In [51]:
# Import libraries
import os, sys
import pandas as pd
import datetime as dt

# spatial libraries
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from fiona.crs import from_epsg
from geopandas import GeoDataFrame

# NYC Open Data API
from sodapy import Socrata

# token
from MyAppToken import key_token

from functools import reduce

# Data preparation
## 311 Data

In [2]:
# import token, username and password for NYC Open Data API
MyAppToken = key_token['token']
username = key_token['username']
password = key_token['password']

In [3]:
client = Socrata("data.cityofnewyork.us",
                 MyAppToken,
                 username=username,
                 password=password)

In [4]:
# SQL filter
path311 = "fhrw-4uyv"
select = 'unique_key,borough,incident_zip,complaint_type,created_date,latitude,longitude'
where = "(complaint_type='HEAT/HOT WATER' OR complaint_type='HEATING') AND (\
        (created_date > '2010-10-01' AND created_date < '2011-05-31') or (created_date > '2011-10-01' AND created_date < '2012-05-31') or \
        (created_date > '2012-10-01' AND created_date < '2013-05-31') or (created_date > '2013-10-01' AND created_date < '2014-05-31') or \
        (created_date > '2014-10-01' AND created_date < '2015-05-31') or (created_date > '2015-10-01' AND created_date < '2016-05-31') or \
        (created_date > '2016-10-01' AND created_date < '2017-05-31') or (created_date > '2017-10-01' AND created_date < '2018-05-31'))"
limit = 2000000

In [5]:
results = client.get(path311, select=select, where=where, limit=limit)

In [22]:
df = pd.DataFrame.from_records(results)
df.shape

(1631064, 7)

In [23]:
# change into datetime format
df['created_date'] = pd.to_datetime(df['created_date'])
df.head()

,borough,complaint_type,created_date,incident_zip,latitude,longitude,unique_key
0,BROOKLYN,HEAT/HOT WATER,2016-03-04 09:25:33,11230,40.62701275833746,-73.95662580149565,32837001
1,MANHATTAN,HEAT/HOT WATER,2016-03-04 10:15:16,10021,40.769015555148385,-73.96321938973729,32837006
2,BRONX,HEAT/HOT WATER,2016-03-04 07:43:24,10460,40.84971553715756,-73.88395060319702,32837007
3,BROOKLYN,HEAT/HOT WATER,2016-03-04 09:54:34,11214,40.59985292883077,-74.0031653164643,32837011
4,MANHATTAN,HEAT/HOT WATER,2016-03-05 17:25:48,10033,40.84687696137512,-73.93600316082807,32837012


In [24]:
# check NaNs
df.isnull().sum()

borough               0
complaint_type        0
created_date          0
incident_zip      11987
latitude          11983
longitude         11983
unique_key            0
dtype: int64

In [25]:
# drop NaNs
df.dropna(inplace=True)
df.shape

(1619077, 7)

In [26]:
# eliminate all zip code outside NYC and change to integer
df['incident_zip'] = df.incident_zip.astype(int)
df = df[df['incident_zip'] < 12000 ]

In [28]:
# set datetime
df['Year'] = df['created_date'].dt.strftime('%Y')
df['YearMonth'] = df['created_date'].dt.strftime('%Y/%m')
df['Month'] = df['created_date'].dt.strftime('%b')

In [45]:
# winter = {}
# for year in list(range(2010,2018)):
#     winter[year] = df[(df['created_date']>dt.date(year,10,1)) & (df['created_date']<dt.date(year+1,5,31))].groupby(['incident_zip']).size().reset_index()

In [139]:
# create new dataframe
winter2017 = df[(df['created_date']>dt.date(2017,10,1)) & (df['created_date']<dt.date(2018,5,31))]
winter2016 = df[(df['created_date']>dt.date(2016,10,1)) & (df['created_date']<dt.date(2017,5,31))]
winter2015 = df[(df['created_date']>dt.date(2015,10,1)) & (df['created_date']<dt.date(2016,5,31))]
winter2014 = df[(df['created_date']>dt.date(2014,10,1)) & (df['created_date']<dt.date(2015,5,31))] 
winter2013 = df[(df['created_date']>dt.date(2013,10,1)) & (df['created_date']<dt.date(2014,5,31))] 
winter2012 = df[(df['created_date']>dt.date(2012,10,1)) & (df['created_date']<dt.date(2013,5,31))] 
winter2011 = df[(df['created_date']>dt.date(2011,10,1)) & (df['created_date']<dt.date(2012,5,31))] 
winter2010 = df[(df['created_date']>dt.date(2010,10,1)) & (df['created_date']<dt.date(2011,5,31))] 

df17 = winter2017.groupby(['incident_zip', 'Month']).size().reset_index()
df16 = winter2016.groupby(['incident_zip', 'Month']).size().reset_index()
df15 = winter2015.groupby(['incident_zip', 'Month']).size().reset_index()
df14 = winter2014.groupby(['incident_zip', 'Month']).size().reset_index()
df13 = winter2013.groupby(['incident_zip', 'Month']).size().reset_index()
df12 = winter2012.groupby(['incident_zip', 'Month']).size().reset_index()
df11 = winter2011.groupby(['incident_zip', 'Month']).size().reset_index()
df10 = winter2010.groupby(['incident_zip', 'Month']).size().reset_index()

In [86]:
# prepare data
dfs = [df10, df11, df12, df13, df14, df15, df16, df17]
for i,d in enumerate(dfs):
    year = 2010
    d.columns = ['zipcode', year+i]
data = reduce(lambda left,right: pd.merge(left,right,on='zipcode'), dfs)
data.head()

,zipcode,2010,2011,2012,2013,2014,2015,2016,2017
0,10001,605,420,287,294,516,455,366,339
1,10002,1369,1014,1353,1126,1439,1181,1488,1607
2,10003,964,840,1104,1031,1230,1249,1203,876
3,10004,15,22,47,12,5,6,10,2
4,10005,6,2,95,9,6,6,7,15


In [142]:
# prepare data
dfs = [df10, df11, df12, df13, df14, df15, df16, df17]

for i,d in enumerate(dfs):
    year = 2010
    d.columns = ['zipcode', 'month', 'heating_complaint']
    d['year'] = year+i

In [143]:
data3 = pd.concat(dfs, ignore_index=True)
data3.shape

(10875, 4)

In [144]:
data3.head()

,zipcode,month,heating_complaint,year
0,10001,Apr,32,2010
1,10001,Dec,138,2010
2,10001,Feb,48,2010
3,10001,Jan,106,2010
4,10001,Mar,58,2010


In [157]:
data3.groupby(['year']).sum().reset_index()

,year,zipcode,heating_complaint
0,2010,14650292,210104
1,2011,14444620,169414
2,2012,14804046,183298
3,2013,14831117,212432
4,2014,14762977,230488
5,2015,14783863,200242
6,2016,14965534,199412
7,2017,14475203,213687


In [97]:
data3.to_csv('heating.csv')

In [64]:
data.set_index('zipcode', inplace=True)
data.to_json('heating.json')

In [99]:
# read geojson neighborhood data
zipcode = "nyc_zip.geojson"
zc = gpd.read_file(zipcode)
zc.shape

(263, 2)

In [100]:
zc.head()

,zipcode,geometry
0,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40..."
1,10463,"(POLYGON ((-73.920646 40.887237, -73.920376 40..."
2,10475,"(POLYGON ((-73.827224 40.89093, -73.825115 40...."
3,10464,"(POLYGON ((-73.815387 40.889394, -73.815265999..."
4,11222,"(POLYGON ((-73.954223 40.739107, -73.954183 40..."


In [120]:
zc['zipcode'] = zc['zipcode'].astype(int)

In [121]:
type(zc['zipcode'][0])

numpy.int64

In [128]:
new = zc.merge(data3, on='zipcode')
new.head()

,zipcode,geometry,heating_complaint,year
0,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40...",244,2010
1,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40...",279,2011
2,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40...",205,2012
3,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40...",416,2013
4,10471,"(POLYGON ((-73.881919 40.906666, -73.878423 40...",294,2014


In [134]:
new.to_file('heat_complaints.geojson', driver="GeoJSON")